In [ ]:
"""
Classifications and Regressions using features generated from the computed mask

Conduct feature selection with the corresponding classificaton model

Conduct featurer selection with the corresponding regression model

"""

In [13]:
# Project-wide Library calls
import nibabel as nib
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from skimage.util import montage 
from skimage.transform import rotate
from sklearn.model_selection import train_test_split as ttt
from sklearn.metrics import accuracy_score as AS
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm import tqdm
from pylab import *
import gc
import torchmetrics 
import radiomics
from torchvision.utils import save_image
import matplotlib.image
import SimpleITK as sitk
import radiomics



In [68]:
FEAT_PATH ='log/feature_extraction/'
tf=torch.load(FEAT_PATH+'tf.pt')
ef=torch.load(FEAT_PATH+'ef.pt')
ttf=torch.load(FEAT_PATH+'ttf.pt')

ts=torch.load(FEAT_PATH+'ts.pt')
es=torch.load(FEAT_PATH+'es.pt')
tts=torch.load(FEAT_PATH+'tts.pt')

tc=torch.load(FEAT_PATH+'tc.pt')
ec=torch.load(FEAT_PATH+'ec.pt')
ttc=torch.load(FEAT_PATH+'ttc.pt')



# remove features with low variance
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=5)
tf=sel.fit_transform(tf)
ef=sel.fit_transform(ef)
ttf=sel.fit_transform(ttf)

tf=torch.tensor(tf)
ef=torch.tensor(ef)
ttf=torch.tensor(ttf)

# combined all data together
F = torch.cat((tf,ef,ttf))

S = torch.cat((ts,es,tts))

C = torch.cat((tc,ec,ttc))

In [233]:

"""

Classificaton: 

Patients are divided into three classes:

short-term: survival days < 10 months
mid-term: 10 months < survival days < 15 months
long-term: survival days > 15 months

"""

x_train,x_test,y_train,y_test=ttt(F,C)


In [234]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif





# Univariate feature selection
f=SelectKBest(f_classif, k=20)

f.fit(x_train,y_train)
x_train=f.transform(x_train)
x_test=f.transform(x_test)

# figure out which feature is selected 
f.get_support()


array([False,  True, False,  True,  True,  True, False, False, False,
        True,  True, False,  True,  True, False, False,  True,  True,
       False, False, False, False, False, False, False,  True, False,
       False,  True, False, False,  True, False, False, False, False,
        True, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
        True, False,  True, False, False, False,  True, False,  True])

In [112]:
# RidgeClassifier feature counts = 4
from sklearn.linear_model import RidgeClassifier

clf = RidgeClassifier().fit(x_train, y_train)
clf.score(x_test,y_test)


0.6086956521739131

In [152]:
# feature counts = 10
from sklearn.svm import SVC

clf = SVC(kernel = 'poly')

clf.fit(x_train,y_train)

pred=clf.predict(x_test)
AS(pred,y_test)


0.6304347826086957

In [209]:
# feature counts =  15
from sklearn.tree import DecisionTreeClassifier as DT
clf = DT()
clf.fit(x_train,y_train)
pred=clf.predict(x_test)
AS(pred,y_test)




0.7391304347826086

In [198]:
# feature counts =  10
from sklearn.neighbors import KNeighborsClassifier as KNN
clf = KNN(n_neighbors=2)
clf.fit(x_train,y_train)
pred=clf.predict(x_test)
AS(pred,y_test)


0.5652173913043478

In [217]:
# feature counts = 15
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=15)
clf.fit(x_train,y_train)
pred=clf.predict(x_test)
AS(pred,y_test)


0.6739130434782609

In [235]:
# feature count = 20 
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier(n_estimators=10)
clf.fit(x_train,y_train)
pred=clf.predict(x_test)
AS(pred,y_test)


0.717391304347826

In [262]:
"""

Regression part

"""
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
x_train,x_test,y_train,y_test=ttt(F,S)

In [263]:
# SKlearn feature extraction and regression
selector = SelectFromModel(estimator=LinearRegression()).fit(x_train, y_train)
print(selector.get_support())
x_train=selector.transform(x_train)
x_test=selector.transform(x_test)

clf=LinearRegression().fit(x_train,y_train)
pred=clf.predict(x_test)
mean_squared_error(pred, y_test)

[ True  True  True False False False False False  True  True False  True
 False False False False False False False False  True False  True  True
 False False  True False False False False False False False False False
 False False  True  True False False False False  True False False False
 False False False False False False False False False False False False
 False False False]


103716.555